# **YOLO object detection with OpenCV**
What is the YOLO object detector?

![대체 텍스트](http://media5.datahacker.rs/2019/04/yolo_diagram-1024x556.jpg)

We can say that the basic idea of the Yolo algorithm is applying both the image classification and localization algorithm on each of nine grid cells. 
![대체 텍스트](http://media5.datahacker.rs/2018/11/025_02.png)



In [ ]:
#@title Import Libraries
# import the necessary packages
import os
import numpy as np
import cv2
import google.colab.patches

# For downloading the image.
import matplotlib.pyplot as plt
import tempfile
import six.moves.urllib.request 
import six 
import PIL

import base64, logging
import io
import IPython.display
import google.colab.output

np.random.seed(42)

In [ ]:
#@title Load data and model
%%shell
mkdir objectDetection; cd objectDetection
mkdir model; cd model
curl -LJO https://github.com/phoebe81/DetectionWithOpenCV/raw/master/Model/coco.names
curl -LJO https://github.com/phoebe81/DetectionWithOpenCV/raw/master/Model/yolov3.cfg
curl -LJO https://github.com/phoebe81/DetectionWithOpenCV/raw/master/Model/yolov3.weights
cd ..; mkdir images; cd images
curl -LJO https://github.com/phoebe81/DetectionWithOpenCV/raw/master/Images/example_01.jpg
curl -LJO https://github.com/phoebe81/DetectionWithOpenCV/raw/master/Images/example_02.jpg
curl -LJO https://github.com/phoebe81/DetectionWithOpenCV/raw/master/Images/example_03.jpg
curl -LJO https://github.com/phoebe81/DetectionWithOpenCV/raw/master/Images/example_04.jpg
curl -LJO https://github.com/phoebe81/DetectionWithOpenCV/raw/master/Images/example_05.jpg
curl -LJO https://github.com/phoebe81/DetectionWithOpenCV/raw/master/Images/example_06.jpg

cd ..; mkdir videos; cd videos
curl -LJO https://github.com/phoebe81/DetectionWithOpenCV/raw/master/Videos/car_chase_01.mp4
cd videos
mkdir save

In [ ]:
# Root directory 
ROOT_DIR = os.path.abspath("./objectDetection/")
# Model directory 
MODEL_DIR = os.path.join(ROOT_DIR, "model")
# Images directory
IMAGE_DIR = os.path.join(ROOT_DIR, "images")

# Video directory
VIDEO_DIR = os.path.join(ROOT_DIR, "videos")
VIDEO_SAVE_DIR = os.path.join(VIDEO_DIR, "save")

In [ ]:
weightsPath = os.path.sep.join([MODEL_DIR, "yolov3.weights"])
configPath = os.path.sep.join([MODEL_DIR, "yolov3.cfg"])
objectNet = cv2.dnn.readNetFromDarknet(configPath, weightsPath)
# objectNet= cv2.dnn.readNet(weightsPath, prototxtPath,"darknet")

In [ ]:
# load the COCO class labels our YOLO model was trained on
labelsPath = os.path.sep.join([MODEL_DIR, "coco.names"])
CLASSES = open(labelsPath).read().strip().split("\n")
COLORS = np.random.randint(0, 255, size=(len(CLASSES), 3),dtype="uint8")

# determine only the *output* layer names that we need from YOLO
# 82,94,106(13 x 13, 26 x 26, 52 x 52) Layer
layer_names = objectNet.getLayerNames()
print('layer name:', layer_names)
output_layers = [layer_names[i - 1] for i in objectNet.getUnconnectedOutLayers()]
print('output_layer name:', output_layers)

# **1**.Detection model demo by picture

In [ ]:
#@title Select an Input Image
SAMPLE_IMAGE = 'example_01.jpg' #@param {type:"string"}
IMAGE_URL = 'https://news.nateimg.co.kr/orgImg/kh/2021/05/20/l_2021052101001947300188761.jpg'  #@param {type:"string"}

def display_image(image):
  fig = plt.figure(figsize=(20, 15))
  plt.grid(False)
  plt.imshow(image)

def download_and_resize_image(url,display=False):
  _, filename = tempfile.mkstemp(suffix=".jpg")
  response = six.moves.urllib.request.urlopen(url)
  image_data = response.read()
  image_data = six.BytesIO(image_data)
  pil_image = PIL.Image.open(image_data)
  pil_image_rgb = pil_image.convert("RGB")
  pil_image_rgb.save(filename, format="JPEG", quality=90)
  print(f"Image downloaded to {filename}.")
  if display:
    display_image(pil_image)
  return filename

if len(IMAGE_URL) !=0 :
  IMAGE_URL = download_and_resize_image(IMAGE_URL)
if len(SAMPLE_IMAGE) !=0 :
  SAMPLE_URL = os.path.join(IMAGE_DIR, SAMPLE_IMAGE)

image_url = IMAGE_URL or SAMPLE_URL

## Apply Detection model

In [ ]:
def detect_and_predict_object(frame,confThreshold=0.5,nmsThreshold=0.4):
  blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (416, 416), swapRB=True, crop=False)
  objectNet.setInput(blob)
  outputs = objectNet.forward(output_layers)
  boxes = []
  confidences = []
  classIDs = []
  for output in outputs:
    for detection in output:
      scores = detection[5:]
      classID = np.argmax(scores)
      confidence = scores[classID]
      
      if confidence > confThreshold:
        box = detection[0:4] * np.array([frame.shape[1], frame.shape[0], frame.shape[1], frame.shape[0]])
        (centerX, centerY, width, height) = box.astype("int")
        
        x = int(centerX - (width / 2))
        y = int(centerY - (height / 2))
         
        boxes.append([x, y, int(width), int(height)])
        confidences.append(float(confidence))
        classIDs.append(classID)
	# apply non-maxima suppression to suppress weak, overlapping bounding boxes
  idxs = cv2.dnn.NMSBoxes(boxes, confidences, confThreshold, nmsThreshold)

	# ensure at least one detection exists
  if len(idxs) > 0:
    for i in idxs.flatten():
      (x, y) = (boxes[i][0], boxes[i][1])
      (w, h) = (boxes[i][2], boxes[i][3])
      
      color = [int(c) for c in COLORS[classIDs[i]]]
      cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
      text = "{}: {:.4f}".format(CLASSES[classIDs[i]], confidences[i])
      cv2.putText(frame, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

In [ ]:
image = cv2.imread(image_url)
detect_and_predict_object(image)
google.colab.patches.cv2_imshow(image)

# **2**.Detection model demo by video

In [ ]:
#@title Videdo Capture
# Using a webcam to capture images for processing on the runtime.
# Source: https://colab.research.google.com/notebooks/snippets/advanced_outputs.ipynb#scrollTo=2viqYx97hPMi

def data_uri_to_img(uri):
  try:
    image = base64.b64decode(uri.split(',')[1], validate=True)
    # make the binary image, a PIL image
    image = PIL.Image.open(io.BytesIO(image))
    # convert to numpy array
    image = np.array(image, dtype=np.uint8);
    return image
  except Exception as e:
    logging.exception(e)
    print('\n')
    return None

def video_to_data_url(filename):
    ext = filename.split('.')[-1]
    prefix = f"data:video/{ext};base64,"
    with open(filename, 'rb') as f:
        vidoe = f.read()
    return prefix + base64.b64encode(vidoe).decode()

# from base64 import b64decode
# playing webcam or video with javascript
def take_photo(filename='photo.jpg', quality=0.8):
  js = IPython.display.Javascript('''     
    async function takePhoto(filename, quality) {
                  
      const div = document.createElement('div');
      document.body.appendChild(div);

      const exit = document.createElement('button');
      exit.textContent = 'Exit';
      div.appendChild(exit);

      const video = document.createElement('video');
      video.style.display = 'block';
           
      if('photo.jpg' == filename){
        const stream = await navigator.mediaDevices.getUserMedia({video: true}); 
        video.srcObject = stream;   
      }else{
        video.src = filename;
        video.type="video/mp4"
      }
      await video.play();  
      div.appendChild(video);       
                   

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);
      
      let jsLog = function(abc) {
        document.querySelector("#output-area").appendChild(document.createTextNode(`${abc}... `));
      }

      // when Exit button is clicked.   
      var isOpened = true; 
      var exitPromise = new Promise((resolve) => {exit.onclick = resolve});   
      exitPromise.then(()=>{isOpened = false; stream.getVideoTracks()[0].stop();});
      
      //when end of video
      var endPromise = new Promise((resolve) => {video.onended = resolve});   
      endPromise.then(()=>{isOpened = false; video.stop();});

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;

      for (let i = 0; isOpened; i++) {
        canvas.getContext("2d").clearRect(0, 0, canvas.width, canvas.height);
        canvas.getContext('2d').drawImage(video, 0, 0);
        img = canvas.toDataURL('image/jpeg', quality);

        // jsLog(i + "sending");
        // Call a python function and send this image
        google.colab.kernel.invokeFunction('notebook.run_objectDetection', [img], {});
        // jsLog(i + "SENT");

        // wait for X miliseconds second, before next capture
        await new Promise(resolve => setTimeout(resolve, 250));        
      }       
      div.remove();      
    }    
    ''')  
  # make the provided HTML, part of the cell
  IPython.display.display(js)
  #call the takePhoto() JavaScript function
  google.colab.output.eval_js(f"takePhoto('{filename}',{quality})")

frame_count = 0
writer = None
# InvokeFunction
# takes the numpy image and runs detection, then shows the results by visualizing
def run_objectDetection(uri): 
  global frame_count, writer

  image = data_uri_to_img(uri)     
  if writer is None:		
      fourcc = cv2.VideoWriter_fourcc(*'DIVX')  
      writer = cv2.VideoWriter(outVideo, fourcc, 2, (image.shape[1], image.shape[0]), True)
  try:        
    detect_and_predict_object(image)

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
    frame_count+=1    
    name = f"{frame_count}.jpg"
    name = os.path.join(VIDEO_SAVE_DIR, name)
    cv2.imwrite(name, image)

    if writer is not None:
      writer.write(image)
  except Exception as e:
    logging.exception(e)
    print('\n')

# register this function, so JS code could call this
google.colab.output.register_callback('notebook.run_objectDetection', run_objectDetection)

## Apply Detection model

In [ ]:
%%shell
cd objectDetection
rm ./videos/save/* 
rm ./videos/out.avi

In [ ]:
inVideo = os.path.join(VIDEO_DIR, "car_chase_01.mp4")
outVideo= os.path.join(VIDEO_DIR, "out.avi")

In [ ]:
frame_count = 0
# data_url = video_to_data_url(inVideo)
data_url = 'photo.jpg'
try: 
  # put the JS code in cell and run it
  take_photo()  
  if writer is not None:
    writer.release()   
except Exception as e:
  logging.exception(e)
  print('\n')

writer = None

## Downlod  video

In [ ]:
from google.colab import files
files.download(outVideo)